# Content Filtering Pipeline

The content filter this pipeline creates is used on our site to recommend movies to users based on movies they select. If a movie has been rated more than three times, though, the collaborative item filter will be applied instead.

## Import Data


In [1]:
import os
import pyodbc
import pandas as pd
from dotenv import load_dotenv

# Load env vars
load_dotenv()

server = os.getenv("AZURE_SQL_SERVER")
database = os.getenv("AZURE_SQL_DATABASE")
username = os.getenv("AZURE_SQL_USERNAME")
password = os.getenv("AZURE_SQL_PASSWORD")

# Build connection string
connection_string = f"""
    DRIVER={{ODBC Driver 17 for SQL Server}};
    SERVER={server},1433;
    DATABASE={database};
    UID={username};
    PWD={password};
    Encrypt=yes;
    TrustServerCertificate=no;
    Connection Timeout=30;
"""

# Connect to SQL Server
conn = pyodbc.connect(connection_string)

# Query to fetch movie data
df = pd.read_sql("SELECT * FROM movies_titles", conn)

# Close connection
conn.close()

# Display the data
display(df.head())

C:\Users\Seank\AppData\Local\Temp\ipykernel_19872\1628476191.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM movies_titles", conn)


,show_id,type,title,director,cast,country,release_year,rating,duration,description,...,Language TV Shows,Musicals,Nature TV,Reality TV,Spirituality,TV Action,TV Comedies,TV Dramas,Talk Shows TV Comedies,Thrillers
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,"Michael Hilow, Ana Hoffman, Dick Johnson, Kirs...",United States,2021,PG-13,90 min,As her father nears the end of his life filmma...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,s2,TV Show,Blood & Water,None,Ama Qamata Khosi Ngema Gail Mabalane Thabang M...,South Africa,2021,TV-MA,2 Seasons,After crossing paths at a party a Cape Town te...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,s3,TV Show,Ganglands,Julien Leclercq,Sami Bouajila Tracy Gotoas Samuel Jouy Nabiha ...,None,2021,TV-MA,1 Season,To protect his family from a powerful drug lor...,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,s4,TV Show,Jailbirds New Orleans,None,None,None,2021,TV-MA,1 Season,Feuds flirtations and toilet talk go down amon...,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,s5,TV Show,Kota Factory,None,Mayur More Jitendra Kumar Ranjan Raj Alam Khan...,India,2021,TV-MA,2 Seasons,In a city of coaching centers known to train I...,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [2]:
df.fillna('Missing', inplace=True)
df.isna().sum()

C:\Users\Seank\AppData\Local\Temp\ipykernel_19872\2904753311.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Missing' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('Missing', inplace=True)


show_id                                               0
type                                                  0
title                                                 0
director                                              0
cast                                                  0
country                                               0
release_year                                          0
rating                                                0
duration                                              0
description                                           0
Action                                                0
Adventure                                             0
Anime Series International TV Shows                   0
British TV Shows Docuseries International TV Shows    0
Children                                              0
Comedies                                              0
Comedies Dramas International Movies                  0
Comedies International Movies                   

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 10)

# Step 1: Import/create a stop word dictionary
## Skip here and create default in step 2

# Step 2: Create a TfidfVectroizer and remove stop words
tfidf = TfidfVectorizer(stop_words='english')

# Step 3: Fit and transform the data into a Tfidf matrix
tfidf_matrix = tfidf.fit_transform(df['description'])

print(tfidf_matrix.shape)

df_tfidf = pd.DataFrame(tfidf_matrix.T.todense(), index=tfidf.get_feature_names_out(), columns=df['description'])
df_tfidf.iloc[1600:1605]
# Converting everything to english is also important for accuracy (this is not done here)

(8513, 18567)


description,As her father nears the end of his life filmmaker Kirsten Johnson stages his death in inventive and comical ways to help them both face the inevitable.,After crossing paths at a party a Cape Town teen sets out to prove whether a private-school swimming star is her sister who was abducted at birth.,To protect his family from a powerful drug lord skilled thief Mehdi and his expert team of robbers are pulled into a violent and deadly turf war.,Feuds flirtations and toilet talk go down among the incarcerated women at the Orleans Justice Center in New Orleans on this gritty reality series.,In a city of coaching centers known to train India’s finest collegiate minds an earnest but unexceptional student and his friends navigate campus life.,The arrival of a charismatic young priest brings glorious miracles ominous mysteries and renewed religious fervor to a dying town desperate to believe.,Equestria's divided. But a bright-eyed hero believes Earth Ponies Pegasi and Unicorns should be pals — and hoof to heart she’s determined to prove it.,On a photo shoot in Ghana an American model slips back in time becomes enslaved on a plantation and bears witness to the agony of her ancestral past.,dasdsa,A talented batch of amateur bakers face off in a 10-week competition whipping up their best dishes in the hopes of being named the U.K.'s best.,A woman adjusting to life after a loss contends with a feisty bird that's taken over her garden — and a husband who's struggling to find a way forward.,"Sicily boasts a bold ""Anti-Mafia"" coalition. But what happens when those trying to bring down organized crime are accused of being criminals themselves?",Struggling to earn a living in Bangkok a man joins an emergency rescue service and realizes he must unravel a citywide conspiracy.,After most of her family is murdered in a terrorist bombing a young woman is unknowingly lured into joining the very group that killed them.,When the clever but socially-awkward Tetê joins a new school she'll do anything to fit in. But the queen bee among her classmates has other ideas.,Cameras following Bengaluru police on the job offer a rare glimpse into the complex and challenging inner workings of four major crime investigations.,"Students of color navigate the daily slights and slippery politics of life at an Ivy League college that's not nearly as ""post-racial"" as it thinks.",Declassified documents reveal the post-WWII life of Otto Skorzeny a close Hitler ally who escaped to Spain and became an adviser to world presidents.,Strangers Diego and Isabel flee their home in Mexico and pretend to be a married couple to escape his drug-dealing enemies and her abusive husband.,After a deadly home invasion at a couple’s new dream house the traumatized wife searches for answers — and learns the real danger is just beginning.,In the 1960s a Holocaust survivor joins a group of self-trained spies who seek justice against Nazis fleeing to Spain to hide after WWII.,In the late 1970s an accused serial rapist claims multiple personalities control his behavior setting off a legal odyssey that captivates America.,When a good deed unwittingly endangers his clan a 13th-century Turkish warrior agrees to fight a sultan's enemies in exchange for new tribal land.,Newly divorced and denied visitation rights with his daughter a doting father disguises himself as a gray-haired nanny in order to spend time with her.,From arcade games to sled days and hiccup cures Cory Carson’s curious little sister Chrissy speeds off on her own for fun and adventure all over town!,When the father of the man she loves insists that his twin sons marry twin sisters a woman creates an alter ego that might be a bit too convincing.,Finding love can be hard for anyone. For young adults on the autism spectrum exploring the unpredictable world of dating is even more complicated.,A tangled love triangle ensues when a man falls for a woman studying to become a nun — and she falls for the friend he enlists to h

The code below generates a table with tons of data and takes a minute or two to run.


In [4]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

df_results = pd.DataFrame(cosine_sim, columns=df['show_id'], index=df['show_id'])

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

df_results = pd.DataFrame(cosine_sim, columns=df['show_id'], index=df['show_id'])
# df_results.to_csv('content_recommendations.csv')
# df_results.sort_values(by=['s8'], ascending=False).head(10)

# Existing transformation code...
df_results.index.name = 'show_id'
df_results = df_results.reset_index()

long_df = df_results.melt(
    id_vars=['show_id'],
    var_name='other_show_id',
    value_name='similarity'
)

long_df = long_df[long_df['show_id'] != long_df['other_show_id']]

top_n = 20
top_matches = (
    long_df.sort_values(['show_id', 'similarity'], ascending=[True, False])
           .groupby('show_id')
           .head(top_n)
           .reset_index(drop=True)
)

display(top_matches)

,show_id,other_show_id,similarity
0,,s7607,1.000000
1,,s7607,1.000000
2,,s7607,1.000000
3,,s772,0.237713
4,,s772,0.237713
...,...,...,...
170155,s999,s6158,0.081789
170156,s999,s1518,0.081539
170157,s999,s3945,0.080573
170158,s999,s1232,0.080566


## Deploy to Azure


In [ ]:
# Establish connection
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

# Create table (replace if exists)
cursor.execute("DROP TABLE IF EXISTS recommender_content")
cursor.execute("""
BEGIN
    CREATE TABLE recommender_content (
        show_id VARCHAR(50),
        other_show_id VARCHAR(50),
        similarity FLOAT
    )
END
""")

# Insert data from the DataFrame (converted to list of tuples)
cursor.fast_executemany = True  # Speed up the inserts for large datasets
cursor.executemany(
    "INSERT INTO recommender_content (show_id, other_show_id, similarity) VALUES (?, ?, ?)",
    top_matches.values.tolist()
)

# Commit and close
conn.commit()
cursor.close()
conn.close()

print("Upload to Azure SQL completed using raw pyodbc!")